In [1]:
import pandas as pd
import numpy as np
import os, sys
import psycopg2 
import sqlalchemy
sys.stdout.flush()

In [2]:
pm = pd.read_csv('/Users/HIT/pm.csv', encoding='utf-8')
print(pm.shape)
pm.head()

(71970, 9)


,obs_date,pvn_nm,bor_nm,so2,co,o3,n02,pm10,pm25
0,20160101,경기도,가평군,0.005000,0.683333,0.007000,0.019792,70.666667,NaN
1,20160102,경기도,가평군,0.006208,0.733333,0.004792,0.024458,71.958333,NaN
2,20160103,경기도,가평군,0.006542,0.820833,0.007625,0.021042,85.208333,NaN
3,20160104,경기도,가평군,0.006917,0.533333,0.024125,0.013833,78.208333,NaN
4,20160105,경기도,가평군,0.006375,0.416667,0.017125,0.014958,38.375000,NaN


## DB 접속

In [3]:
# Connect to database (Note: The package psychopg2 is required for Postgres to work with SQLAlchemy)
engine = sqlalchemy.create_engine("postgresql://analytics:analytics!@#@localhost:5432/postgres")
con = engine.connect()
table_name = 'pm_test'

### 지역 정보 가져오기

In [4]:
import pandas as pd
import numpy as np
import os, sys
import psycopg2 
import sqlalchemy
sys.stdout.flush()

conn_string = "host='localhost' dbname ='postgres' user='analytics' password='analytics!@#'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

sql = """
		select distinct case when pvn_nm  = '인천광역시' then pvn_nm || ' '||bor_nm 
		                     when bor_nm in ('수원시권선구', '성남시중원구', '고양시덕양구', '안산시상록구') then substring(bor_nm,1,3) || ' ' || substring(bor_nm,4,6)
		else bor_nm end 
		from contest.pm_test
"""

cur.execute(sql)
result = cur.fetchall()


### 지역수 확인

In [5]:
### check 
print(len(result))
print("Sample %s " % result[0][0])

66
Sample 인천광역시 서구 


In [7]:
# DB session close
con.close()

## 지역별 점포 수 크롤링

In [8]:
from selenium import webdriver
from lxml import html, etree, cssselect
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

default = 'http://lalavla.gsretail.com/lalavla/ko/market-info'
count = 0 
first_crawl = []
# 크롤링할 사이트 호출
# 첫 페이지
for i in range(len(result)) :
    if count == 0:
        driver = webdriver.Chrome("C:/Users/HIT/Desktop/Default_Path/chromedriver.exe")
        driver.get(default)
        driver.find_element_by_css_selector('#stb3').send_keys(result[i][0])
        driver.implicitly_wait(10)
        driver.find_element_by_css_selector("#stb3").send_keys(Keys.RETURN)
        driver.implicitly_wait(10)
        # Idle Time
        time.sleep(2)
        
        doc = html.fromstring(driver.page_source)
        element = doc.xpath('//*[@id="searchList"]/tbody')[0].text_content().split('랄라블라')
        first_crawl.append(len(element)-1) 
        count += 1
        # Idle Time
        time.sleep(2)
        
# 1이상 페이지
    else :
        driver.get(default)
        
        driver.find_element_by_css_selector('#stb3').send_keys(result[i][0])
        driver.implicitly_wait(10)
        driver.find_element_by_css_selector("#stb3").send_keys(Keys.RETURN)
        driver.implicitly_wait(10)
        # Idle Time
        time.sleep(2)
        
        doc = html.fromstring(driver.page_source)
        element = doc.xpath('//*[@id="searchList"]/tbody')[0].text_content().split('랄라블라')
        first_crawl.append(len(element)-1) 
        count += 1
        # Idle Time
        time.sleep(2)
        

driver.quit()

In [18]:
local_name = [result[i][0] for i in range(len(result))]
np.transpose(pd.DataFrame([local_name, first_crawl])).head()

,0,1
0,인천광역시 서구,1
1,인천광역시 강화군,0
2,양평군,0
3,인천광역시 옹진군,5
4,광주시,0


In [19]:
pd.DataFrame(result).to_csv('/Users/HIT/Desktop/store.csv', encoding='euc-kr')

## promotion (Lalavla)

In [21]:
from selenium import webdriver
from lxml import html, etree, cssselect
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

default = 'http://lalavla.gsretail.com/lalavla/ko/customer-engagement/event/last-events'
nextPageJs = 'lastEvent.movePage'
count = 0 
first_crawl = []
# 크롤링할 사이트 호출
# 첫 페이지
for i in range (1, 33) :
    if count == 0:
        driver = webdriver.Chrome("C:/Users/HIT/Desktop/Default_Path/chromedriver.exe")
        driver.get(default)
        doc = driver.page_source
        element = html.fromstring(doc)
        first_crawl.append([element.xpath('//*[@id="contents"]/div[2]/div/div/div/table')[0].text_content().split('\t\t\t\t')])
        count += 1
        driver.execute_script('lastEvent.movePage'+'(' + str(i+1) + ')')
        # Idle Time
        time.sleep(2)
    else :
        doc = driver.page_source
        element = html.fromstring(doc)
        first_crawl.append([element.xpath('//*[@id="contents"]/div[2]/div/div/div/table')[0].text_content().split('\t\t\t\t')])
        count += 1
        driver.execute_script('lastEvent.movePage'+'(' + str(i+1) + ')')
        # Idle Time
        time.sleep(2)
        

driver.quit()

## 프로모션 데이터 정제

In [22]:
second_phase = []
for i in range (len(first_crawl)) :
    try :
        second_phase.append(first_crawl[i][0][39:])
    except :
        pass

In [23]:
# DataFrame 화
channel, promotion, date = [], [], []
#  x = 10 , y= 32
for x in range (10) :
    for y in range(32) :
        channel.append(pd.DataFrame(second_phase)[x].apply(lambda s : s.split('\t\t'))[y][0])
        promotion.append(pd.DataFrame(second_phase)[x].apply(lambda s : s.split('\t\t'))[y][1])
        date.append(pd.DataFrame(second_phase)[x].apply(lambda s : s.split('\t\t'))[y][2])
print('channel rows %s, promotion name %s, promotion date %s' % (len(channel), len(promotion), len(date)))

channel rows 320, promotion name 320, promotion date 320


## DataFrame 화

In [24]:
gs_promotion = pd.DataFrame({'channel': channel, 'promotion' : promotion, 'date' : date })
gs_promotion['str_date'] = gs_promotion.date.apply(lambda x : x.replace('기간 : ','')).apply(lambda x : x.split('~')[0])
gs_promotion['end_date'] = gs_promotion.date.apply(lambda x : x.replace('기간 : ','')).apply(lambda x : x.split('~')[1])

print('Check DF')
print('DF rows : %s, cols : %s' % (gs_promotion.shape))

Check DF
DF rows : 320, cols : 5


In [25]:
gs_promotion[(gs_promotion.str_date >= '2016.01.01') & (gs_promotion.end_date <= '2018.12.31')]

,channel,promotion,date,str_date,end_date
0,온라인,가을 무드에 빠지다! 세일 폴폴~,기간 : 2019.09.06 ~ 2019.09.13,2019.09.06,2019.09.13
1,온라인,호캉스템 vs 홈캉스템 뷰티 패키지 투어 최대 50%OFF,기간 : 2019.07.04 ~ 2019.07.08,2019.07.04,2019.07.08
2,오프라인,"랄라블라,현대카드 H-Coin 100%사용 이벤트!",기간 : 2019.04.29 ~ 2019.09.11,2019.04.29,2019.09.11
3,온라인,달리프&어헤즈 브랜드데이 헤어 BEST 대전,기간 : 2019.04.11 ~ 2019.04.14,2019.04.11,2019.04.14
4,온라인,문샷 코렉트핏 쿠션 뷰티토커 모집,기간 : 2019.01.31 ~ 2019.02.17,2019.01.31,2019.02.17
5,온라인,편의점 픽업하고 리틀 프렌즈 받고!,기간 : 2018.12.12 ~ 2019.01.10,2018.12.12,2019.01.10
6,온라인,"부르조아 럭키박스, 만원의 행복",기간 : 2018.11.01 ~ 2018.11.30,2018.11.01,2018.11.30
7,오프라인,원더 랄라블라데이!,기간 : 2018.10.11 ~ 2018.10.15,2018.10.11,2018.10.15
8,온라인,홈페이지 단독 이벤트 이지듀 정품 득템 기회!,기간 : 2018.08.02 ~ 2018.08.12,2018.08.02,2018.08.12
9,오프라인,현대카드 H-Coin 이벤트 1만 H-Coin 사용시 데메테르 디퓨저를 드립니다.,기간 : 2018.06.11 ~ 2018.07.08,2018.06.11,2018.07.08


## 최종 데이터 확인

In [26]:
gs_fix = gs_promotion[gs_promotion['channel'] == '오프라인']
gs_fix.str_date = gs_fix.str_date.apply(lambda x : x.replace('.',''))
gs_fix.end_date = gs_fix.end_date.apply(lambda x : x.replace('.',''))

col = ['channel', 'promotion', 'str_date', 'end_date']
gs_fix = gs_fix[col]
print('Check DF')
print('DF rows : %s, cols : %s' % (gs_fix.shape))
gs_fix.head()

Check DF
DF rows : 161, cols : 4


C:\Users\HIT\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,channel,promotion,str_date,end_date
2,오프라인,"랄라블라,현대카드 H-Coin 100%사용 이벤트!",20190429,20190911
7,오프라인,원더 랄라블라데이!,20181011,20181015
9,오프라인,현대카드 H-Coin 이벤트 1만 H-Coin 사용시 데메테르 디퓨저를 드립니다.,20180611,20180708
10,오프라인,스무살 뷰티 꽃길 랄라블라 데이,20180517,20180522
11,오프라인,lalavla 정회원이라면?! 대리주부 할인쿠폰 쏜다!!,20180329,20180415


## Export 

In [27]:
gs_fix.to_csv('/Users/HIT/Desktop/gsPromotion.csv', encoding='utf-8')